In [6]:
%pip install bitsandbytes

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/101.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/101.6 MB 650.2 kB/s eta 0:02:37
   ---------------------------------------- 0.3/101.6 MB 2.5 MB/s eta 0:00:42
    --------------------------------------- 2.1/101.6 MB 12.3 MB/s eta 0:00:09
   - -------------------------------------- 2.9/101.6 MB 13.1 MB/s eta 0:00:08
   - -------------------------------------- 3.4/101.6 MB 13.4 MB/s eta 0:00:08
   - -------------------------------------- 3.7/101.6 MB 13.3 MB/s eta 0:00:08
   - -------------------------------------- 3.7/101.6 MB 13.3 MB/s eta 0:00:08
   - -------------------------------------- 3.7/101.6 MB 13.3 MB/s eta 0:00:08
   - -------------------------------------- 4.3/101.6 MB 9.8 MB/s eta 0:00:10
   -- ------------------------------------- 5.2/101.6 MB 10.6 MB/s eta 0:00

In [11]:
%pip install accelerate

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/309.4 kB ? eta -:--:--
   --------- ------------------------------ 71.7/309.4 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 309.4/309.4 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install peft

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [15]:
%pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/116.9 kB ? eta -:--:--
     -------------------------------------- 116.9/116.9 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/244.2 kB ? eta -:--:--
   ---------------------------------------- 244.2/244.2 kB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.9 kB ? eta -:--:--
   ---------------------------------------- 72.9/72.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/92.5 MB ? eta -:--:--
    --------------------------------------- 1.2/92.5 MB 18.2 MB/s eta 0:00:06
   - -------------------------------------- 2.7/92.5 MB 21.5 MB/s eta 0:00:05
   - -------------------------------------- 3.3/92.5 MB 19.3 MB/s eta 0:00:05
   - -------------------------------------- 4.1/92.5 MB 18.7 MB/s eta 0:00:05
   - -------------------------------------- 4.4/92.5 MB 17.5 MB/s eta 0:00:06
   - -----------------

  You can safely remove it manually.


In [1]:
import numpy as np 
import pandas as pd 
import torch 
import transformers
from time import time


c:\Users\satwi\anaconda3\envs\llama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torch import cuda, bfloat16
model_id = r"C:\Users\satwi\Downloads\HTM\CompanionAI\merged_model"

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)
# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16, 
)

cuda:0


In [3]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='cuda:0',
)  
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Loading checkpoint shards: 100%|██████████| 3/3 [00:35<00:00, 11.91s/it]


Prepare model, tokenizer: 47.246 sec.


In [4]:
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto"
)

In [5]:
time_start = time()
input_text = 'I am feeling depresred'

# Generate sequences
sequences = query_pipeline(
    input_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

# Print the generated sequences
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\satwi\anaconda3\envs\llama\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Result: I am feeling depresred and anxious. everybody tells me that i should just focus on the positive things in my life, but it is hard to do that when i am feeling so down. i am trying to get help, but it is hard to find the right person to talk to. i am feeling so alone and lost right now.

It sounds like you are going through a difficult time and are struggling with some heavy emotions. It's important to remember that you are not alone in this feeling, and that many people experience depression and anxiety at some point in their lives. It's okay to feel overwhelmed and lost, and it's important to reach out for support.

One way to find support is to talk to a mental health professional, such as a therapist or counselor. They can provide a safe and non-judgmental space for you to talk about your feelings and work through them. You can find
Prepare model, tokenizer: 59.388 sec.


In [6]:
input_text = 'I think i am worthless i should just kill myself'

# Generate sequences
sequences = query_pipeline(
    input_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

# Print the generated sequences
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


Result: I think i am worthless i should just kill myself. Unterscheidung between the two, the first step in overcoming the feeling of worthlessness is to identify the underlying cause.
I have a friend who has been struggling with feelings of worthlessness for a long time. He has been seeing a therapist for a few months now, but the feelings still persist. He has been feeling hopeless and helpless for a long time, and he is starting to feel like he is worthless.
Feeling worthless can be a very painful and distressing experience, and it is important to seek help if you are struggling with these feelings. If you are feeling worthless, it is important to remember that you are not alone and that there are many resources available to help you cope with these feelings.
If you are feeling worthless, it is important to reach out to a trusted friend, family member, or mental health professional for support. Talk


In [7]:
time_start = time()
input_text = 'I want to commit suicide'

# Generate sequences
sequences = query_pipeline(
    input_text,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

# Print the generated sequences
for seq in sequences:
    print(f"Result: {seq['generated_text']}")
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Result: I want to commit suicide. nobody can help me. i am so depressed and hopeless. i don't want to be here anymore. i am 15 years old.

I'm so sorry to hear that you're feeling this way. It's important to know that you are not alone, and there are people who care about you and want to help. Suicide is a very serious topic, and I'm glad you reached out for help.

First, please know that you are not alone. Many people feel sad, hopeless, or like they want to hurt themselves, and it's okay to feel this way. It's important to talk to someone you trust about how you're feeling, like a parent, teacher, or counselor. They can help you find ways to feel better and stay safe.

If you're in immediate danger, please call emergency services or go to the nearest
Prepare model, tokenizer: 67.691 sec.
